In [ ]:
import pandas as pd
from datetime import datetime
import sources.utils, sources.classes, sources.accounting, sources.io_utils
import importlib
importlib.reload(sources.utils)
importlib.reload(sources.classes)
importlib.reload(sources.accounting)
importlib.reload(sources.io_utils)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 100)

In [ ]:
#TODO Sign up for Etherscan / Arbiscan and get access tokens. Set them in "io_utils.py"

In [ ]:

eth_address = "0xa2e11fA386C698E525185EF211472555cDF006C3"
#todo set INITIAL_DEPOSIT_WALLET in accounting.py (the wallet with source funds)

time_start = datetime(2018, 1, 1, 0,0,0)
time_end = datetime.now()
today_string = time_end.strftime("%Y-%m-%d")

In [ ]:

data_dir = "./data/"
etherscan_dfs = sources.io_utils.get_or_load_etherscan_dfs(time_start, time_end, data_dir, eth_address, explorer="etherscan")
arbiscan_dfs = sources.io_utils.get_or_load_etherscan_dfs(time_start, time_end, data_dir, eth_address, explorer="arbiscan")

In [ ]:
# this can take some time because it tries to get method name of contract. 
# TODO: add option to not get method name. But then must adapt Tx classifier to not use method name
ethereum_tx_df = sources.io_utils.combine_etherscan_dfs(etherscan_dfs, eth_address)
arbitrum_tx_df = sources.io_utils.combine_etherscan_dfs(arbiscan_dfs, eth_address, platform="arbitrum")

In [ ]:
tx_df = pd.concat([ethereum_tx_df, arbitrum_tx_df], ignore_index=True)
tx_df.sort_values("TimeStamp", inplace=True)

tokens = tx_df[["TokenName","TokenSymbol"]].copy().drop_duplicates()
tokens = sources.io_utils.match_tokens_to_coingecko(tokens)

cg_ids = tokens[tokens["cg_id"].notnull()]["cg_id"].unique()
prices_df = sources.io_utils.fetch_historical_prices(cg_ids, time_start, time_end)
token_id2current_price = sources.utils.get_token_id2current_price(prices_df, tokens, time_end)

tx_df_priced = sources.utils.merge_tx_df_with_prices(tx_df, tokens, prices_df)

In [ ]:
portfolio, tx_df_gains = sources.accounting.compute_portfolio_and_gains(tx_df_priced)